In [ ]:
import pandas as pd

Format EIA Data

In [ ]:
# The EIA data table is formatted with notes at the beginning and end, drop them from dataframe
eiaDropHead = [0, 1]
eiaDropTail = [23698, 23699]

# EIA list of tabs to merge
eiaTabs = ['Operating', 'Planned', 'Retired', 'Canceled or Postponed', 'Operating_PR', 'Planned_PR', 'Retired_PR']

for i in eiaTabs:
    eia = pd.read_excel(r'C:\Users\Sarah\Documents\GEM\research\North America\US\EIA\july_generator2021.xlsx', sheet_name=i, skiprows=eiaDropHead)
    eia = eia.drop(eiaDropTail)

    # Keep only gas-fired plants
    eia = eia.loc[eia['Energy Source Code'] == 'NG']

    # Add status column for the tab
    eia['Status'] = t.lower()

    # Combine tabs


In [ ]:
print(eia)

In [ ]:
# EIA QC
print(eia.columns)
print(eia['Energy Source Code'].unique())

In [ ]:
# Merge units into proper sets

# Match IDs to GGPT and compare field values (status, capacity, year)

# Locate any new unit IDs

# Check capacity for any new units

In [ ]:
# Add in ownership information from separate EIA table